In [1]:
import pandas as pd

In [2]:
# !for i in `ls ./`;do echo experiment_name = \"$i\";done

In [3]:
# experiment_name = "2023.12.20-1_Mistral_7B_instruct_API_3-few-shots_RANDOM"
# experiment_name = "2023.12.20-2_Mistral_7B_instruct_API_5-few-shots_RANDOM"
# experiment_name = "2023.12.20-3_Mistral_7B_instruct_API_7-few-shots_RANDOM"
# experiment_name = "2023.12.20-4_Mistral_7B_instruct_API_3-few-shots_SEMANTIC_SIMILARITY"
# experiment_name = "2023.12.20-5_Mistral_7B_instruct_API_5-few-shots_SEMANTIC_SIMILARITY"
# experiment_name = "2023.12.20-6_Mistral_7B_instruct_API_7-few-shots_SEMANTIC_SIMILARITY"
# experiment_name = "2023.12.20-7_Mistral_7B_instruct_API_3-few-shots_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES"
# experiment_name = "2023.12.20-8_Mistral_7B_instruct_API_5-few-shots_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES"
experiment_name = "2023.12.20-9_Mistral_7B_instruct_API_7-few-shots_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES"

In [4]:
!ls -l ./data/output/$experiment_name/ | head -5

total 4416
-rw-r--r-- 1 jovyan users   5771 Feb 17 13:44 2023.12.20-9_Mistral_7B_instruct_API_7-few-shots_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES_classification_report.xlsx
-rw-r--r-- 1 jovyan users 335040 Jan  3 12:39 report_sdg_classification_Mistral_7B.xlsx
-rw-r--r-- 1 jovyan users    418 Dec 20 15:01 sdg_classification_Mistral_7B_instruct_API_0.pkl
-rw-r--r-- 1 jovyan users    422 Dec 20 15:25 sdg_classification_Mistral_7B_instruct_API_1000.pkl
ls: write error: Broken pipe


In [5]:
output_datapath = f"./data/output/{experiment_name}/"

In [6]:
results_processed_merged = pd.read_excel(output_datapath+'report_sdg_classification_Mistral_7B.xlsx')

In [7]:
results_processed_merged.head(3)

,Unnamed: 0,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 0---,0
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 11---,11
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---,16


In [8]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       1.00      0.38      0.55        60
           1       0.54      0.88      0.67        60
           2       0.94      0.52      0.67        60
           3       0.86      0.85      0.86        60
           4       0.83      0.83      0.83        60
           5       0.85      0.68      0.76        60
           6       0.98      0.68      0.80        60
           7       0.88      0.63      0.74        60
           8       0.43      0.77      0.55        60
           9       0.67      0.67      0.67        60
          10       0.56      0.33      0.42        60
          11       0.56      0.92      0.70        60
          12       0.58      0.95      0.72        60
          13       0.81      0.78      0.80        60
          14       0.93      0.83      0.88        60
          15       0.84      0.63      0.72        60
          16       0.86      0.83      0.85        60

    accuracy    

In [9]:
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if x>16 else x)

In [10]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE 

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
# sdg_list = [str(x) for x in range(17)]
sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list 
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['predicted_label_clean']) or row['predicted_label_clean'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'predicted_label_clean'] = 0  # new_value

In [11]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       1.00      0.38      0.55        60
           1       0.54      0.88      0.67        60
           2       0.94      0.52      0.67        60
           3       0.86      0.85      0.86        60
           4       0.83      0.83      0.83        60
           5       0.85      0.68      0.76        60
           6       0.98      0.68      0.80        60
           7       0.88      0.63      0.74        60
           8       0.43      0.77      0.55        60
           9       0.67      0.67      0.67        60
          10       0.56      0.33      0.42        60
          11       0.56      0.92      0.70        60
          12       0.58      0.95      0.72        60
          13       0.81      0.78      0.80        60
          14       0.93      0.83      0.88        60
          15       0.84      0.63      0.72        60
          16       0.86      0.83      0.85        60

    accuracy    

In [12]:
# from io import StringIO

# classification_report_str = classification_report(y_true, y_pred, digits=3)
# df = pd.read_csv(StringIO(classification_report_str), sep='\s+', engine='python')
# df[['precision', 'recall', 'f1-score']] *= 100
# df

In [13]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [14]:
report_df

,precision,recall,f1-score,support
0,100.000000,38.333333,55.421687,60.000000
1,54.081633,88.333333,67.088608,60.000000
2,93.939394,51.666667,66.666667,60.000000
3,86.440678,85.000000,85.714286,60.000000
4,83.333333,83.333333,83.333333,60.000000
5,85.416667,68.333333,75.925926,60.000000
6,97.619048,68.333333,80.392157,60.000000
7,88.372093,63.333333,73.786408,60.000000
8,42.990654,76.666667,55.089820,60.000000
9,66.666667,66.666667,66.666667,60.000000


In [15]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [16]:
print(precision, recall, accuracy, f1_score)

77.23410895657565 71.66666666666667 71.66666666666667 71.64890626416494


In [17]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

77.2 71.7 71.7 71.6
